In [ ]:
import pandas as pd
import numpy as np
import keras
import os
import re
import math
import shutil
import matplotlib.pyplot as plt

from google.colab import drive



from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,recall_score,precision_score,f1_score

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer


from keras.metrics import AUC,Recall,Precision,BinaryAccuracy
import keras.backend as K
from keras.layers import Hashing
from keras.utils import pad_sequences
tf.get_logger().setLevel('ERROR')

In [ ]:

# changes directory from colab to gdrive
drive.mount('/content/drive', force_remount=True)

base_path = '/content/drive/MyDrive/INFO251_Fake_Job_Detection/'
os.chdir(base_path)
baseline_csv = './Data/text.csv'  # data dictionary
data = os.path.join(base_path, baseline_csv)
dfRaw = pd.read_csv(data)

Mounted at /content/drive


# Read in Data

In [ ]:
dfRaw = pd.read_csv('text.csv')

## Train-test Split

In [ ]:
train_test_split(dfRaw,test_size=0.25,random_state=42)[0]

,text,fraudulent
13016,provide full time permanent positions many med...,f
15068,established principles full time education eve...,f
3147,recombine provides clinical genetic testing ca...,f
9157,european dynamics leading european software in...,f
5224,working cubical seems like idea hell joining a...,f
...,...,...
11284,hayes corp create fun stuff focus software dev...,f
11964,mission making cities smarter cities complex s...,f
5390,qb medical inc medical supply distributor loca...,f
860,storesupport offer great working environment f...,f


In [ ]:
labels=np.array(dfRaw.fraudulent.map({'t':1,'f':0}).to_list()).reshape((-1,1))
X_train,X_test,y_train,y_test=train_test_split(dfRaw.text,labels,test_size=0.25,random_state=42)


# Bert Model

In [ ]:

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

## Define my BERT Class for taining and testing

In [ ]:
class MyBERT():
  def __init__(self,bert_model_name='small_bert/bert_en_uncased_L-4_H-512_A-8'):
    self.bert_model_name = bert_model_name
    
    self.tfhub_handle_encoder = map_name_to_handle[self.bert_model_name]
    self.tfhub_handle_preprocess = map_model_to_preprocess[self.bert_model_name]

    print(f'BERT model selected           : {self.tfhub_handle_encoder}')
    print(f'Preprocess model auto-selected: {self.tfhub_handle_preprocess}')
    self.bert_preprocess_model = hub.KerasLayer(self.tfhub_handle_preprocess, name='preprocessing')
    self.bert_model = hub.KerasLayer(self.tfhub_handle_encoder, trainable=True, name='BERT_encoder')

  def build_classifier_model(self,r_dropout=0.1):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = self.bert_preprocess_model
    encoder_inputs = preprocessing_layer(text_input)
    encoder = self.bert_model
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(r_dropout)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

  def fit(self,X_train,y_train,class_wight=[1.0,1.0]):
    def F1Score(y_true, y_pred): #taken from old keras source code
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
      predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
      precision = true_positives / (predicted_positives + K.epsilon())
      recall = true_positives / (possible_positives + K.epsilon())
      f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
      return f1_val

    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metric = [AUC(),Recall(),Precision(),BinaryAccuracy(),F1Score]
    epochs = 15
    steps_per_epoch = len(X_train)
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')

    self.classifier_model = self.build_classifier_model()
    self.classifier_model.compile(optimizer=optimizer,
                            loss=loss,
                            metrics=metric)

    print(f'Training model with {self.tfhub_handle_encoder}')
    history = self.classifier_model.fit(x=X_train,y=y_train,
                                  validation_split=0.2,
                                  class_weight={0:class_wight[0],1:class_wight[1]},
                                  epochs=epochs)
    return history

  def eval(self,X_test,y_test):
    return self.classifier_model.evaluate(x=X_test,y=y_test)

  def save(self,hyperparam):
    saved_model_path = './Model/BERT_MODEL/{}_bert'.format('_'.join(hyperparam))
    self.classifier_model.save(saved_model_path, include_optimizer=False)
    print(f'model {saved_model_path} saved!')

## Get ready for hyper-parameter tunning and general training

In [ ]:
dfModelCompare = pd.DataFrame(
    columns=['name','model','dropout_r','class_0','class_1','loss','auc','recall','precision','accu','f1']
    )
models = ['small_bert/bert_en_uncased_L-4_H-512_A-8',
          'small_bert/bert_en_uncased_L-8_H-512_A-8',
          'bert_en_uncased_L-12_H-768_A-12']
dropout_r = [0.1,0.2,0.3,0.5,0.6]
class_weight = [[1.0,1.0],[1.0,1.5],[1.0,2.0],[1.0,3.0],[1.0,5.0]]

In [ ]:
cnt = 0
for m in models:
  model = MyBERT(bert_model_name=m)
  for r in dropout_r:
    model.build_classifier_model(r)
    for w in class_weight:
      if cnt<8:
        cnt += 1
        continue
      model.fit(X_train,y_train,w)
      results = model.eval(X_test,y_test)
      model.save([str(cnt),str(r),str(w[0]),str(w[1])])
      cnt+=1
      dfModelCompare.loc[len(dfModelCompare.index)] = [cnt,m,r]+w+results
      # Record training results
      dfModelCompare.to_csv('Model_Compare.csv')


# Read in the best model so far and do prediction and evaluation

In [ ]:
def F1Score(y_true, y_pred): #taken from old keras source code
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  recall = true_positives / (possible_positives + K.epsilon())
  f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
  return f1_val
best_model = './Model/BERT_MODEL/7_0.2_1.0_2.0_bert'
reloaded_model = tf.keras.saving.load_model(best_model,custom_objects={'F1Score':F1Score})


In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = [AUC(),Recall(),Precision(),BinaryAccuracy(),F1Score]
epochs = 15
steps_per_epoch = len(X_train)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                      num_train_steps=num_train_steps,
                      num_warmup_steps=num_warmup_steps,
                      optimizer_type='adamw')

reloaded_model.compile(optimizer=optimizer,
            loss=loss,
            metrics=metric)
reloaded_model.evaluate(x=X_test,y=y_test)

140/140 [==============================] - 395s 3s/step - loss: 0.1133 - auc: 0.9090 - recall: 0.8206 - precision: 0.9433 - binary_accuracy: 0.9886 - F1Score: 0.6935


[0.11328911036252975,
 0.909018874168396,
 0.8206278085708618,
 0.9432989954948425,
 0.9885905981063843,
 0.693500280380249]